In [19]:
import boto3
import time
import base64

bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")

def call_nova_use_s3_file(s3_uri, model_id, prompt, system_prompt, temperature, top_p, max_token):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "video": {
                        "format": "mp4",
                        "source": {
                            "s3Location": {
                                "uri": s3_uri
                            }
                        }
                    }
                },
                {"text": prompt},
            ],
        }
    ]

    system = [{
        "text": system_prompt
    }]

    inferenceConfig = {
        "maxTokens": int(max_token),
        'temperature': temperature,
        'topP': top_p
    }

    start_time = time.time()

    response = bedrock_runtime.converse(
        modelId=model_id,
        messages=messages,
        system=system,
        inferenceConfig=inferenceConfig
    )

    # 计算耗时
    elapsed_time = time.time() - start_time
    print(f"API调用耗时: {elapsed_time:.2f}秒")
    return response

def call_nova_use_local_file(video_local_path, model_id, prompt, system_prompt, temperature, top_p, max_token):
    with open(video_local_path, "rb") as file:
        media_bytes = file.read()

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "video": {
                        "format": "mp4",
                        "source": {
                            "bytes": media_bytes
                        }
                    }
                },
                {"text": prompt},
            ],
        }
    ]

    system = [{
        "text": system_prompt
    }]

    inferenceConfig = {
        "maxTokens": int(max_token),
        'temperature': temperature,
        'topP': top_p
    }

    start_time = time.time()

    response = bedrock_runtime.converse(
        modelId=model_id,
        messages=messages,
        system=system,
        inferenceConfig=inferenceConfig
    )

    # 计算耗时
    elapsed_time = time.time() - start_time
    print(f"API调用耗时: {elapsed_time:.2f}秒")
    return response

In [20]:
s3_uri = 's3://allenzh-public/video_understanding/the-sea.mp4'
model_id = 'us.amazon.nova-pro-v1:0'
# model_id = 'arn:aws:bedrock:us-west-2:096331270838:inference-profile/us.amazon.nova-pro-v1:0'

# Provide a clear, descriptive prompt in English
prompt = "Analyze this video and describe what you see. Focus on the main events and important details."

# Provide a clear system prompt that guides the model's behavior
system_prompt = "You are a video analysis assistant. Provide detailed, objective descriptions of the video content."

# Adjust parameters within acceptable ranges
temperature = 0.1  # Between 0 and 1
top_p = 0.3       # Between 0 and 1
max_token = 512   # Typically between 1 and 2048

response = call_nova_use_s3_file(s3_uri, model_id, prompt, system_prompt, temperature, top_p, max_token)
print(response.get("output"))
print(response.get("usage"))


API调用耗时: 4.23秒
{'message': {'role': 'assistant', 'content': [{'text': 'The video starts with a top-down view of a rocky coastline with a large rock formation in the middle of the ocean. The camera then pans to the right, showing a large seashell on the beach. The seashell is brown and white, with a spiral pattern. The waves are crashing on the shore, and the sun is setting in the background.'}]}}
{'inputTokens': 3495, 'outputTokens': 72, 'totalTokens': 3567}
